In [ ]:
## BERT

In [ ]:
!pip install transformers torch

In [ ]:
import pandas as pd

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
import torch

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

In [ ]:
from sklearn.preprocessing import LabelEncoder, label_binarize

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from itertools import cycle

In [ ]:
import numpy as np

In [ ]:
# Encoding labels ('positive', 'negative', 'neutral') to numerical values

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df['label'] = label_encoder.fit_transform(df['class'])

In [ ]:
# Tokenization and Data Preparation (using cleaned text)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
encoded_data = tokenizer.batch_encode_plus(

In [ ]:
    df['cleaned_text'].tolist(),

In [ ]:
    add_special_tokens=True,

In [ ]:
    return_attention_mask=True,

In [ ]:
    padding='max_length',

In [ ]:
    max_length=128,  # Adjust if your sentences are longer

In [ ]:
    truncation=True,

In [ ]:
    return_tensors='pt'

In [ ]:
)

In [ ]:
input_ids = encoded_data['input_ids']

In [ ]:
attention_masks = encoded_data['attention_mask']

In [ ]:
labels = torch.tensor(df['label'].tolist())

In [ ]:
# Split into training and testing sets

In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(

In [ ]:
    input_ids, labels, random_state=35476648, test_size=0.2

In [ ]:
)

In [ ]:
train_masks, test_masks, _, _ = train_test_split(

In [ ]:
    attention_masks, input_ids, random_state=35476648, test_size=0.2

In [ ]:
)

In [ ]:
# Create DataLoader

In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)

In [ ]:
test_dataloader = DataLoader(test_data, batch_size=32)

In [ ]:
# Load Pre-trained Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3) # 3 labels for positive, negative, neutral

In [ ]:
# Enable GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

In [ ]:
# Fine-tune (Optimizer, Loss Function, Training Loop)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(3):  # Adjust number of epochs

In [ ]:
    model.train()

In [ ]:
    for batch in train_dataloader:

In [ ]:
        batch = tuple(t.to(device) for t in batch)

In [ ]:
        b_input_ids, b_input_mask, b_labels = batch

In [ ]:
        optimizer.zero_grad()

In [ ]:
        outputs = model(b_input_ids,

In [ ]:
                        token_type_ids=None,

In [ ]:
                        attention_mask=b_input_mask)

In [ ]:
        loss = loss_fn(outputs.logits, b_labels)

In [ ]:
        loss.backward()

In [ ]:
        optimizer.step()

In [ ]:
# Evaluation

In [ ]:
model.eval()

In [ ]:
logits_list, true_labels = [], []

In [ ]:
for batch in test_dataloader:

In [ ]:
    batch = tuple(t.to(device) for t in batch)

In [ ]:
    b_input_ids, b_input_mask, b_labels = batch

In [ ]:
    with torch.no_grad():

In [ ]:
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

In [ ]:
    logits = outputs.logits.detach().cpu().numpy()

In [ ]:
    label_ids = b_labels.to('cpu').numpy()

In [ ]:
    logits_list.extend(logits)

In [ ]:
    true_labels.extend(label_ids)

In [ ]:
# Convert logits to probabilities for ROC AUC

In [ ]:
softmax = torch.nn.Softmax(dim=1)

In [ ]:
probs = softmax(torch.tensor(logits_list)).numpy()

In [ ]:
# Binarize the labels for ROC AUC calculation

In [ ]:
y_test_binarized = label_binarize(true_labels, classes=[0, 1, 2])

In [ ]:
n_classes = y_test_binarized.shape[1]

In [ ]:
fpr, tpr, roc_auc = {}, {}, {}

In [ ]:
for i in range(n_classes):

In [ ]:
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], probs[:, i])

In [ ]:
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# Calculate macro-average ROC AUC

In [ ]:
roc_auc["macro"] = roc_auc_score(y_test_binarized, probs, multi_class="ovr", average="macro")

In [ ]:
# Plot ROC AUC curves

In [ ]:
plt.figure()

In [ ]:
colors = cycle(['blue', 'green', 'red'])

In [ ]:
for i, color in zip(range(n_classes), colors):

In [ ]:
    plt.plot(fpr[i], tpr[i], color=color, lw=2,

In [ ]:
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

In [ ]:
plt.plot([0, 1], [0, 1], 'k--', lw=2)

In [ ]:
plt.xlim([0.0, 1.0])

In [ ]:
plt.ylim([0.0, 1.05])

In [ ]:
plt.xlabel('False Positive Rate')

In [ ]:
plt.ylabel('True Positive Rate')

In [ ]:
plt.title('Multiclass ROC')

In [ ]:
plt.legend(loc="lower right")

In [ ]:
plt.show()

In [ ]:
# Print out all AUC scores

In [ ]:
print("AUC for each class:")

In [ ]:
for i in range(n_classes):